In [2]:
import torch   

### 1.1 GRPO KL散度

GRPO KL是token-level的

In [ ]:
def grpo_kl(pi_logprob,pi_ref_logprob):
    """
    GRPO KL散度
    """
    return torch.exp(pi_logprob - pi_ref_logprob) - (pi_logprob - pi_ref_logprob) - 1

pi = torch.randn(3,5) # batch, sequence11
pi_ref = torch.randn(3,5) # batch, sequence
pi_logprob = torch.nn.functional.log_softmax(pi, dim = 1)
pi_ref_logprob = torch.nn.functional.log_softmax(pi_ref, dim =1)
print(grpo_kl(pi_logprob, pi_ref_logprob))

tensor([[1.3747e+00, 6.2129e-02, 1.4377e-02, 6.8426e+01, 4.1986e+00],
        [1.1180e-01, 2.4026e-01, 2.0036e+01, 2.0704e+00, 8.4948e-01],
        [7.6509e-02, 1.6087e+00, 3.2334e-01, 9.8950e-03, 1.5139e+00]])


### 1.2 GRPO损失

In [ ]:

def grpo_loss(pi_logprob,pi_old_logprob,pi_ref_logprob,advantage,input_len,len_oi):
    """
    GRPO损失函数

    pi_logprob: 当前策略 π 的 log 概率，形状 [bs, seq_len]
    pi_old_logprob: 旧策略 π_old 的 log 概率，形状 [bs, seq_len]
    pi_ref_logprob: 参考策略 π_ref 的 log 概率（比如 SFT 模型或基线），形状 [bs, seq_len]
    advantage: 每个样本的优势值，形状 [bs]
    input_len: prompt 的长度（int），也就是从哪个位置开始是模型的 response token
    len_oi: 每条样本的输出长度（output length / len_of_interest），通常是 response 的长度，用来做长度归一化.
    """
    
    # 1.超参数设定
    epsilon = 0.2 # 超参数，用于控制 KL 散度的惩罚力度
    beta = 0.01 # KL 惩罚项的权重，控制「新策略不要离 ref 太远」。权重越大，惩罚越重。

    # 2.基本形状 & 构建长度向量
    bs,seq_len = pi_logprob.shape
    # skip计算采样的每条样本的输出长度
    len_oi = torch.tensor([len_oi]*group_num,dtype=torch.float32)
    # 这里把一个标量 len_oi 复制成长度为 group_num 的向量。
    # 通常 group_num 是一组样本的数量，比如 GRPO 会把多个候选 response 视为一组。
    # 这步的目的：为后面对每个样本做长度归一化 1/len_oi 做准备。

    # 3.构造 mask：只在 response 段算 loss
    mask = torch.zeros(bs,seq_len)
    # prompt 部分 [ : , :input_len ] = 0
    # response 部分 [ : , input_len: ] = 1
    # 这样一来，后面 loss * mask 就只在 response token 上起作用，prompt 的 logprob 不参与更新。
    mask[:,input_len:] = 1

    # 4.PPO 风格的 ratio & clipped ratio
    ratio = torch.exp(pi_logprob - pi_old_logprob)
    # ratio_clip 把它裁剪到 [1-ε, 1+ε] 范围里，防止更新太大（PPO 的核心技巧）。
    ratio_clip = torch.clamp(ratio,1- epsilon,1+epsilon)

    # 5. 广播 advantage，构造 policy gradient 项
    advantage = advantage.unsqueeze(dim=1) # [bs] -> [bs,1]
    policy_gradient = torch.minium(ratio*advantage,ratio_clip*advantage) # [bs,seq_len]

    # 6. KL 正则项（相对参考策略）
    # 不希望当前策略偏离 ref 模型太远，防止训练走偏（和 PPO 里针对旧策略的 KL 惩罚类似，只不过这里是对 reference model）。
    kl = grpo_kl(pi_logprob,pi_ref_logprob)

    # 7. 把 policy gradient 和 KL 合在一起，并用 mask 选 token
    loss = (policy_gradient - beta*kl) * mask

    # 8. 按组数 + 长度归一化，并取负号形成「loss」
    # 8.1 group_num 通常指“每个 prompt 产生了多少个候选 response”（GRPO 里的 group size）。
        # 除以 group_num 相当于：同一个 prompt 的多个 sample 一起平均，让 loss 的规模不随 group size 改变。
    
    # 8.2 假设 len_oi 形状是 [bs]，表示每条样本的 有效 response 长度（len of interest）。
    # unsqueeze(dim=1) 把它变成 [bs, 1]，在 token 维自动广播到 [bs, seq_len]：
    # 举例: 
    # len_oi = [10, 20, 30]
    # len_oi.unsqueeze(dim=1) = [[10], [20], [30]]
    # 1/ len_oi.unsqueeze(dim=1) = [[1/10], [1/20], [1/30]]
    # 1/ len_oi.unsqueeze(dim=1) 在 token 维自动广播到 [bs, seq_len]：
    # 1/ len_oi.unsqueeze(dim=1) = [[1/10, 1/10, 1/10, 1/10, 1/10, 1/10, 1/10, 1/10, 1/10, 1/10],
    #                              [1/20, 1/20, 1/20, 1/20, 1/20, 1/20, 1/20, 1/20, 1/20, 1/20],
    #                              [1/30, 1/30, 1/30, 1/30, 1/30, 1/30, 1/30, 1/30, 1/30, 1/30]]

    loss = (-1 / group_num) * (1/ len_oi.unsqueeze(dim=1)) * loss
    loss = loss.sum()

    return loss


测试

In [ ]:
# 输出分布
pi_logits = torch.randn(3,5,32) # batch, seq_len, vocab_size
pi_ref_logits = torch.randn(3,5,32)
pi_old_logits = torch.randn(3,5,32)

# 获取log prob
pi_logprob = F.log_softmax(pi_logits, dim = -1)
pi_ref_logprob = F.log_softmax(pi_ref_logits, dim = -1)
pi_old_logprob = F.log_softmax(pi_old_logits, dim = -1)

# group data
token_ids = torch.tensor([[11,12,13,14,15], # 输入为11,12,13, 输出为:14, 15
                          [11,12,13,15,16],
                          [11,12,13,16,17],])

# 获取policy
pi_logprob = torch.gather(pi_logprob, dim=-1, index=token_ids.unsqueeze(-1)).squeeze(-1)
pi_ref_logprob = torch.gather(pi_ref_logprob, dim=-1, index=token_ids.unsqueeze(-1)).squeeze(-1)
pi_old_logprob = torch.gather(pi_old_logprob, dim=-1, index=token_ids.unsqueeze(-1)).squeeze(-1)
loss = grpo_loss(pi_logprob, pi_old_logprob, pi_ref_logprob, A,3,2)
print(loss)

### 1.3 完整可运行版本

In [ ]:
import torch
import torch.nn.functional as F

def grpo_kl(pi_logprob, pi_ref_logprob):
    """
    GRPO KL散度近似（token-level）

    pi_logprob, pi_ref_logprob: [bs, seq_len]
    返回同形状的 KL 值：
        kl = exp(delta) - delta - 1
        其中 delta = log pi - log pi_ref,  r = pi/ref
        这个形式对应 f(r) = r - log r - 1
    """
    delta = pi_logprob - pi_ref_logprob
    return torch.exp(delta) - delta - 1  # [bs, seq_len]


def grpo_loss(pi_logprob, pi_old_logprob, pi_ref_logprob, advantage, input_len, len_oi):
    """
    GRPO损失函数

    pi_logprob: 当前策略 π 的 log 概率，形状 [bs, seq_len]
    pi_old_logprob: 旧策略 π_old 的 log 概率，形状 [bs, seq_len]
    pi_ref_logprob: 参考策略 π_ref 的 log 概率（比如 SFT 模型或基线），形状 [bs, seq_len]
    advantage: 每个样本的优势值，形状 [bs]
    input_len: prompt 的长度（int），也就是从哪个位置开始是模型的 response token
    len_oi: 每条样本的输出长度（可以是标量或 [bs]），用于长度归一化
    """
    # 使用外部的 group_num
    global group_num

    # 1. 超参数设定
    epsilon = 0.2  # PPO clip 范围
    beta = 0.01    # KL 惩罚项的权重

    # 2. 基本形状
    bs, seq_len = pi_logprob.shape
    device = pi_logprob.device
    dtype = pi_logprob.dtype

    # 2.1 处理 len_oi：允许传标量或 tensor
    if not torch.is_tensor(len_oi):
        # 标量 -> 每条样本都用同一个长度
        len_oi = torch.tensor([len_oi] * bs, dtype=torch.float32, device=device)
    else:
        len_oi = len_oi.to(device=device, dtype=torch.float32)
        if len_oi.ndim == 0:
            len_oi = len_oi.repeat(bs)  # 标量张量 -> [bs]
        elif len_oi.shape[0] != bs:
            raise ValueError(f"len_oi 的长度 {len_oi.shape[0]} 必须等于 batch_size {bs}")
    len_oi = len_oi.clamp_min(1.0)  # 防止除以 0

    # 3. 构造 mask：只在 response 段算 loss
    mask = torch.zeros(bs, seq_len, device=device, dtype=dtype)
    mask[:, input_len:] = 1.0
    # prompt 部分 [ : , :input_len ] = 0
    # response 部分 [ : , input_len: ] = 1

    # 4. PPO 风格的 ratio & clipped ratio
    ratio = torch.exp(pi_logprob - pi_old_logprob)              # [bs, seq_len]
    ratio_clip = torch.clamp(ratio, 1 - epsilon, 1 + epsilon)   # [bs, seq_len]

    # 5. 广播 advantage，构造 policy gradient 项
    advantage = advantage.unsqueeze(dim=1)                      # [bs] -> [bs,1]
    policy_gradient = torch.minimum(ratio * advantage,
                                    ratio_clip * advantage)     # [bs, seq_len]

    # 6. KL 正则项（相对参考策略）
    kl = grpo_kl(pi_logprob, pi_ref_logprob)                    # [bs, seq_len]

    # 7. 把 policy gradient 和 KL 合在一起，并用 mask 选 token
    loss = (policy_gradient - beta * kl) * mask                 # [bs, seq_len]

    # 8. 按组数 + 长度归一化，并取负号形成「loss」
    loss = (-1.0 / group_num) * (1.0 / len_oi.unsqueeze(dim=1)) * loss
    loss = loss.sum()   # 标量

    return loss


# ================== 测试代码，可直接运行 ==================

# 先测 grpo_kl
pi = torch.randn(3, 5)       # [batch, seq]
pi_ref = torch.randn(3, 5)
pi_logprob = F.log_softmax(pi, dim=1)
pi_ref_logprob = F.log_softmax(pi_ref, dim=1)
# 这是 token 级别的 KL 近似
print("KL tokens example:\n", grpo_kl(pi_logprob, pi_ref_logprob))

# 从完整分布中提取特定 token 的概率。
# 模拟输出分布 logits
pi_logits = torch.randn(3, 5, 32)       # [batch, seq_len, vocab_size]
pi_ref_logits = torch.randn(3, 5, 32)
pi_old_logits = torch.randn(3, 5, 32)

# 获取 log prob over vocab
# 计算全词汇 log 概率：在最后一个维度（dim=-1）上做 log_softmax，得到每个位置对所有词汇的 log 概率。
# [batch, seq_len, vocab_size]
pi_logprob_all = F.log_softmax(pi_logits, dim=-1)
pi_ref_logprob_all = F.log_softmax(pi_ref_logits, dim=-1)
pi_old_logprob_all = F.log_softmax(pi_old_logits, dim=-1)

# group data: 每条序列的实际 token id
# 定义实际 token 序列：token_ids 表示每个样本在每个位置实际生成的 token ID
token_ids = torch.tensor([
    [11, 12, 13, 14, 15],  # 输入为11,12,13, 输出为14,15
    [11, 12, 13, 15, 16],
    [11, 12, 13, 16, 17],
])

# 从 logits 里取出对应 token 的 log prob → [bs, seq_len]

# token_ids.unsqueeze(-1)：将 [3, 5] 扩展为 [3, 5, 1]，以匹配 gather 的索引形状要求。
# gather(dim=-1, ...)：在最后一个维度上按索引收集，从每个位置的 32 个概率中取出对应 token 的概率
# squeeze(-1)：将 [3, 5, 1] 压缩回 [3, 5]
pi_logprob = torch.gather(pi_logprob_all, dim=-1, index=token_ids.unsqueeze(-1)).squeeze(-1)
pi_ref_logprob = torch.gather(pi_ref_logprob_all, dim=-1, index=token_ids.unsqueeze(-1)).squeeze(-1)
pi_old_logprob = torch.gather(pi_old_logprob_all, dim=-1, index=token_ids.unsqueeze(-1)).squeeze(-1)

# 定义 group_num 和 advantage
group_num = 3                      # 这里 bs=3，假设一组就3个候选
A = torch.randn(3)                 # advantage: [bs]
input_len = 3                      # 前 3 个是 prompt，后 2 个是 response
len_out = 2                        # 每条样本 response 长度 = 2

loss = grpo_loss(pi_logprob, pi_old_logprob, pi_ref_logprob, A, input_len, len_out)
print("GRPO loss:", loss)


KL tokens example:
 tensor([[3.8230e+01, 6.7101e-02, 2.3538e+00, 6.5938e-02, 2.6931e-01],
        [4.7228e+00, 5.5592e-01, 1.5142e+00, 2.2383e+00, 1.1279e+00],
        [3.1912e-02, 9.4552e-02, 7.5410e-02, 3.4297e-01, 7.1797e+00]])
GRPO loss: tensor(0.6428)
